In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
df = pd.read_csv("data/fer2013.csv")
df.head()

In [ ]:
emotions_dict = {0:"Angry", 1:"Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5:"Surprise", 6:"Neutral"}
#0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

In [ ]:
# Combining PublicTest and PrivateTest to the whole testing set
df_training = df[df["Usage"] == "Training"]
df_validation = df[df["Usage"] == "PublicTest"]
df_testing = df[df["Usage"] == "PrivateTest"]

In [ ]:
# Save training set and testing set as csv file for the purpose of futher use
# Uncomment commands below if you need to save these files
# df_training.to_csv("data/train.csv", index = False)
# df_validation.to_csv("data/test.csv", index = False)
# df_testing.to_csv("data/validation.csv", index = False)

In [ ]:
# create inputs and labels for the training set
def get_inputs_labels(file_path):
    df = pd.read_csv(file_path)
    inputs = [np.array(df["pixels"][i].split(" ")).astype(int) for i in range(df.shape[0])]
    labels = [df["emotion"][i] for i in range(df.shape[0])]
    assert len(inputs) == len(labels), "inputs and labels should have the same length."
    return np.array(inputs), np.array(labels)

In [ ]:
# Data Augmentation and Preprocessing
def training_testing_prep(training_file_path, validation_file_path, testing_file_path):
    image_size = 48
    training_inputs1, training_labels = get_inputs_labels(training_file_path)
    validation_inputs1, validation_labels = get_inputs_labels(validation_file_path)
    testing_inputs1, testing_labels = get_inputs_labels(testing_file_path)
    
    training_inputs = training_inputs1.reshape(training_inputs1.shape[0], image_size, image_size, 1)
    ver_lst = [[]]
    for i in range(len(training_inputs)):
        flip = [list(np.concatenate(training_inputs[i,::-1,:,0]))]
        ver_lst += flip
    ver_ary = np.array(ver_lst[1:])
    training_inputs1 = np.concatenate((training_inputs1, ver_ary), axis = 0)
    training_inputs = training_inputs1.reshape(training_inputs1.shape[0], image_size, image_size, 1)

    
    validation_inputs = validation_inputs1.reshape(validation_inputs1.shape[0], image_size, image_size, 1)
    ver_lst = [[]]
    for i in range(len(validation_inputs)):
        flip = [list(np.concatenate(validation_inputs[i,::-1,:,0]))]
        ver_lst += flip
    ver_ary = np.array(ver_lst[1:])
    validation_inputs1 = np.concatenate((validation_inputs1, ver_ary), axis = 0)
    validation_inputs = validation_inputs1.reshape(validation_inputs1.shape[0], image_size, image_size, 1)
    
    
    testing_inputs = testing_inputs1.reshape(testing_inputs1.shape[0], image_size, image_size, 1)
    ver_lst = [[]]
    for i in range(len(testing_inputs)):
        flip = [list(np.concatenate(testing_inputs[i,::-1,:,0]))]
        ver_lst += flip
    ver_ary = np.array(ver_lst[1:])
    testing_inputs1 = np.concatenate((testing_inputs1, ver_ary), axis = 0)
    testing_inputs = testing_inputs1.reshape(testing_inputs1.shape[0], image_size, image_size, 1)
    
    training_labels = tf.one_hot(training_labels, 7, dtype=tf.float32)
    training_labels = tf.convert_to_tensor(np.concatenate((training_labels, training_labels), axis = 0), dtype=tf.float32)
    
    validation_labels = tf.one_hot(validation_labels, 7, dtype=tf.float32)
    validation_labels = tf.convert_to_tensor(np.concatenate((validation_labels, validation_labels), axis = 0), dtype=tf.float32)
    
    testing_labels = tf.one_hot(testing_labels, 7, dtype=tf.float32)
    testing_labels = tf.convert_to_tensor(np.concatenate((testing_labels, testing_labels), axis = 0), dtype=tf.float32)
    
    return training_inputs, training_labels, validation_inputs, validation_labels, testing_inputs, testing_labels

In [ ]:
training_inputs, training_labels, validation_inputs, validation_labels, testing_inputs, testing_label = training_testing_prep("data/train.csv", "data/validation.csv", "data/test.csv")

In [ ]:
plt.figure(0, figsize=(20,10))
for i in range(1, 13):
    plt.subplot(4,3,i)
    plt.imshow(training_inputs[i, :, :, 0], cmap="gray")
    plt.text(60, 25, emotions_dict[list(training_labels[i].numpy()).index(1.0)], ha = "center")

In [ ]:
plt.figure(0, figsize=(20,10))
for i in range(1, 13):
    plt.subplot(4,3,i)
    plt.imshow(training_inputs[28709 + i, :, :, 0], cmap="gray")
    plt.text(60, 25, emotions_dict[list(training_labels[28709 + i].numpy()).index(1.0)], ha = "center")